In [1]:
import numpy as np
import subprocess
import os
import cv2

In [2]:
# # use ffmpeg to convert the video to a series of images
# def extract_frames(video_path, output_dir, duration=30):
#     # Create output directory if it doesn't exist
#     os.makedirs(output_dir, exist_ok=True)

#     # Run ffmpeg command to extract frames
#     command = [
#         'ffmpeg',
#         '-i', video_path,
#         # '-vf', 'fps=1/1',
#         '-t', str(duration),
#         os.path.join(output_dir, 'frame_%04d.png')
#     ]
#     subprocess.run(command)

# video_path = './ForrestGump.mp4'
# output_dir = 'frames'
# extract_frames(video_path, output_dir)

In [3]:
# use viola jones to detect faces in the images
def detect_faces(image_path):
    # Load the cascade
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Read the input image
    img = cv2.imread(image_path)

    # Convert into grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray)

    # Draw rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the output
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

detect_faces('frames/frame_0001.png')
